In [18]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'Iris.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
iris = pd.read_csv(file_content_stream)
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [19]:
# Frequency table of species
iris['Species'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

In [20]:
# changing labels of numbers 
iris['species_numb'] = np.where(iris['Species'] == 'Iris-virginica', 1, np.where(iris['Species'] == 'Iris-versicolor', 2, 3))

In [21]:
# defining inputs and target 
x = iris.drop(columns = ['Id', 'Species', 'species_numb'])
y = iris['species_numb']

# splitting the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [22]:
#scaling the data
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [23]:
# building random forest model 
one_vs_all_rf = OneVsRestClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(x_train, y_train)
#predicting on the test
one_vs_all_rf_pred = one_vs_all_rf.predict_proba(x_test)
one_vs_all_rf_pred = np.argmax(one_vs_all_rf_pred, axis = 1) + 1

print(confusion_matrix(y_test, one_vs_all_rf_pred))
print(classification_report(y_test, one_vs_all_rf_pred))

[[ 8  2  0]
 [ 0 10  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       1.00      0.80      0.89        10
           2       0.83      1.00      0.91        10
           3       1.00      1.00      1.00        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



In [25]:
# building random forest model 
one_vs_all_svm = OneVsRestClassifier(estimator = SVC(kernel = 'rbf', C = 0.1, probability = True)).fit(x_train, y_train)
#predicting on the test
one_vs_all_svm_pred = one_vs_all_svm.predict_proba(x_test)
one_vs_all_svm_pred = np.argmax(one_vs_all_svm_pred, axis = 1) + 1

print(confusion_matrix(y_test, one_vs_all_svm_pred))
print(classification_report(y_test, one_vs_all_svm_pred))

[[ 8  2  0]
 [ 1  9  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       0.89      0.80      0.84        10
           2       0.82      0.90      0.86        10
           3       1.00      1.00      1.00        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30



In [ ]:
# based on these results i would use random forest because it has the better performace